# 16-02: 代码覆盖率与质量工具ESLint、Oxlint 和代码覆盖率分析。

In [ ]:
// ========== 1. ESLint 配置 ==========
// eslint.config.mjs (Flat Config)
// npm install --save-dev eslint @typescript-eslint/parser @typescript-eslint/eslint-plugin
/*
import eslint from '@eslint/js';
import tseslint from 'typescript-eslint';
import globals from 'globals';

export default tseslint.config(
  eslint.configs.recommended,
  ...tseslint.configs.recommendedTypeChecked,
  {
    languageOptions: {
      parserOptions: {
        project: './tsconfig.json',
        tsconfigRootDir: import.meta.dirname,
      },
      globals: {
        ...globals.node,
      },
    },
    rules: {
      // TypeScript 严格规则
      '@typescript-eslint/no-explicit-any': 'error',
      '@typescript-eslint/no-unused-vars': ['error', { argsIgnorePattern: '^_' }],
      '@typescript-eslint/explicit-function-return-type': 'warn',
      '@typescript-eslint/no-floating-promises': 'error',
      '@typescript-eslint/await-thenable': 'error',
      
      // 代码风格
      'prefer-const': 'error',
      'no-var': 'error',
      'eqeqeq': ['error', 'always'],
      'curly': ['error', 'all'],
    },
  },
  {
    files: ['**/*.test.ts', '**/*.spec.ts'],
    rules: {
      '@typescript-eslint/no-explicit-any': 'off',
    },
  },
);
*/
// 命令:
// npx eslint .
// npx eslint . --fix (自动修复)

In [ ]:
// ========== 2. Oxlint 配置 ==========
// Oxlint 是 Rust 编写的高性能 JavaScript/TypeScript 检查器
// npm install --save-dev oxlint
// .oxlintrc.json
/*
{
  "extends": [
    "plugin:unicorn/recommended"
  ],
  "rules": {
    "no-console": "warn",
    "no-debugger": "error",
    "prefer-const": "error"
  },
  "ignorePatterns": [
    "dist/**",
    "node_modules/**",
    "**/*.test.ts"
  ]
}
*/
// 命令:
// npx oxlint .
// npx oxlint . --fix

// Oxlint 比 ESLint 快 50-100 倍，适合做 CI 快速检查

In [ ]:
// ========== 3. TypeScript 严格配置 ==========
// tsconfig.json
/*
{
  "compilerOptions": {
    "target": "ES2022",
    "module": "NodeNext",
    "moduleResolution": "NodeNext",
    "lib": ["ES2022"],
    
    // 严格模式
    "strict": true,
    "noImplicitAny": true,
    "strictNullChecks": true,
    "strictFunctionTypes": true,
    "strictBindCallApply": true,
    "strictPropertyInitialization": true,
    "noImplicitThis": true,
    "alwaysStrict": true,
    
    // 额外严格检查
    "noUnusedLocals": true,
    "noUnusedParameters": true,
    "noImplicitReturns": true,
    "noFallthroughCasesInSwitch": true,
    "noUncheckedIndexedAccess": true,
    "exactOptionalPropertyTypes": true,
    
    // 模块解析
    "esModuleInterop": true,
    "skipLibCheck": true,
    "forceConsistentCasingInFileNames": true,
    
    // 输出
    "declaration": true,
    "declarationMap": true,
    "sourceMap": true,
    "outDir": "./dist",
    "rootDir": "./src"
  },
  "include": ["src/**/*"],
  "exclude": ["node_modules", "dist", "**/*.test.ts"]
}
*/

In [ ]:
// ========== 4. Vitest 覆盖率配置 ==========
// vitest.config.ts
/*
import { defineConfig } from 'vitest/config';

export default defineConfig({
  test: {
    coverage: {
      provider: 'v8', // 或 'istanbul'
      reporter: ['text', 'json', 'html'],
      reportsDirectory: './coverage',
      
      // 阈值设置
      thresholds: {
        lines: 80,
        functions: 80,
        branches: 70,
        statements: 80
      },
      
      // 排除文件
      exclude: [
        'node_modules/',
        'dist/',
        '**/*.d.ts',
        '**/*.test.ts',
        '**/types.ts',
        '**/index.ts'
      ],
      
      // 包含文件
      include: ['src/**/*']
    }
  }
});
*/
// 命令:
// npx vitest run --coverage

In [ ]:
// ========== 5. 覆盖率报告解读 ==========
/*
Coverage Report:
----------------|---------|----------|---------|---------|-------------------
File            | % Stmts | % Branch | % Funcs | % Lines | Uncovered Line #s
----------------|---------|----------|---------|---------|-------------------
All files       |   85.23 |    72.45 |   88.00 |   84.92 |
 src            |   85.23 |    72.45 |   88.00 |   84.92 |
  utils.ts      |   92.00 |    85.00 |   90.00 |   91.67 | 15-20
  auth.ts       |   78.00 |    60.00 |   85.00 |   76.00 | 45-60,88
----------------|---------|----------|---------|---------|-------------------

指标说明:
- % Stmts (Statements): 语句覆盖率 - 被执行的语句百分比
- % Branch (Branches): 分支覆盖率 - if/else, switch 等分支的覆盖
- % Funcs (Functions): 函数覆盖率 - 被调用的函数百分比
- % Lines (Lines): 行覆盖率 - 被执行的代码行百分比

建议阈值:
- 语句覆盖率: >= 80%
- 分支覆盖率: >= 70%
- 函数覆盖率: >= 80%
- 行覆盖率: >= 80%

关键文件应达到 90%+*/

In [ ]:
// ========== 6. Git Hooks (Husky) ==========
// npm install --save-dev husky lint-staged
// npx husky init
// .husky/pre-commit
/*
#!/bin/sh
. "$(dirname "$0")/_/husky.sh"

# 运行 lint-staged
npx lint-staged

# 运行类型检查
npx tsc --noEmit

# 运行测试
npx vitest run --coverage
*/
// package.json
/*
{
  "lint-staged": {
    "*.{ts,tsx}": [
      "oxlint --fix",
      "prettier --write",
      "eslint --fix"
    ],
    "*.json": ["prettier --write"],
    "*.md": ["prettier --write"]
  }
}
*/